In [ ]:
from utils import *
check_tensorflow_gpu()

proxy = SpatiotemporalCO2()

proxy.num_epochs = 100
proxy.batch_size = 15
proxy.monitor_cb = 10
proxy.lr_cb      = 20
proxy.loss_alpha = 0.25

In [ ]:
proxy.load_data()
proxy.process_data()

In [ ]:
model = proxy.make_model()
fit   = proxy.train()